<a href="https://colab.research.google.com/github/aliakbarbadri/persian-poetry-creator/blob/master/char-gru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

# Load the data

In [0]:
url = "https://raw.githubusercontent.com/aliakbarbadri/persian-poetry-creator/master/shahname2.txt"
filepath = keras.utils.get_file("shahname.txt", url) 
corpus = open(filepath, 'rb').read().decode(encoding='utf-8')

In [3]:
print(len(corpus),"chars")

2554745 chars


In [4]:
print(corpus[:110])

چو از بلخ بامی به جیحون رسید	سپهدار لشکر فرود آورید
بشد شهریار از میان سپاه	فرود آمد از باره بر شد به گاه
بخوا


In [5]:
vocab = sorted(set(corpus))
print ('{} unique characters'.format(len(vocab)))

48 unique characters


In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in corpus])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\t':   0,
  '\n':   1,
  ' ' :   2,
  '(' :   3,
  ')' :   4,
  '«' :   5,
  '»' :   6,
  '،' :   7,
  '؟' :   8,
  'ء' :   9,
  'آ' :  10,
  'أ' :  11,
  'ؤ' :  12,
  'ئ' :  13,
  'ا' :  14,
  'ب' :  15,
  'ت' :  16,
  'ث' :  17,
  'ج' :  18,
  'ح' :  19,
  ...
}


In [8]:
seq_length = 1000
examples_per_epoch = len(corpus)//(seq_length+1)

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(10):
  print(idx2char[i.numpy()])

چ
و
 
ا
ز
 
ب
ل
خ
 


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'چو از بلخ بامی به جیحون رسید\tسپهدار لشکر فرود آورید\nبشد شهریار از میان سپاه\tفرود آمد از باره بر شد به گاه\nبخواند او گرانمایه جاماسپ را\tکجا رهنمون بود گشتاسپ را\nسر موبدان بودو شاه ردان\tچراغ بزرگان و اسپهبدان\nچنان پاک تن بود و تابنده جان\tکه بودی بر او آشکارا نهان\nستاره\u200cشناس و گرانمایه بود\tابا او به دانش کرا پایه بود\nبپرسید ازو شاه و گفتا خدای\tترا دین به داد و پاکیزه رای\nچو تو نیست اندر جهان هیچ کس\tجهاندار دانش ترا داد و بس\nببایدت کردن ز اختر شمار\tبگویی همی مر مرا روی کار\nکه چون باشد آغاز و فرجام جنگ\tکرا بیشتر باشد اینجا درنگ\nنیامد خوش آن پیر جاماسپ را\tبه روی دژم گفت گشتاسپ را\nکه میخواستم کایزد دادگر\tندادی مرا این خرد وین هنر\nمرا گر نبودی خرد شهریار\tنکردی زمن بودنی خواستار\nمگر با من از داد پیمان کند\tکه نه بد کند خود نه فرمان کند\nجهانجوی گفتا به نام خدای\tبدین و به دین آور پاک رای\nبه جان زریر آن نبرده سوار\tبه جان گرانمایه اسفندیار\nکه نه هرگزت روی دشمن کنم\tنفرمایمت بد نه خود من کنم\nتو هرچ اندرین کار دانی بگوی\tکه تو چاره\u200cدانی و من چاره\u200cجوی\nخ

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
  print (repr(''.join(idx2char[input_example.numpy()])))
  print (repr(''.join(idx2char[target_example.numpy()])))

'چو از بلخ بامی به جیحون رسید\tسپهدار لشکر فرود آورید\nبشد شهریار از میان سپاه\tفرود آمد از باره بر شد به گاه\nبخواند او گرانمایه جاماسپ را\tکجا رهنمون بود گشتاسپ را\nسر موبدان بودو شاه ردان\tچراغ بزرگان و اسپهبدان\nچنان پاک تن بود و تابنده جان\tکه بودی بر او آشکارا نهان\nستاره\u200cشناس و گرانمایه بود\tابا او به دانش کرا پایه بود\nبپرسید ازو شاه و گفتا خدای\tترا دین به داد و پاکیزه رای\nچو تو نیست اندر جهان هیچ کس\tجهاندار دانش ترا داد و بس\nببایدت کردن ز اختر شمار\tبگویی همی مر مرا روی کار\nکه چون باشد آغاز و فرجام جنگ\tکرا بیشتر باشد اینجا درنگ\nنیامد خوش آن پیر جاماسپ را\tبه روی دژم گفت گشتاسپ را\nکه میخواستم کایزد دادگر\tندادی مرا این خرد وین هنر\nمرا گر نبودی خرد شهریار\tنکردی زمن بودنی خواستار\nمگر با من از داد پیمان کند\tکه نه بد کند خود نه فرمان کند\nجهانجوی گفتا به نام خدای\tبدین و به دین آور پاک رای\nبه جان زریر آن نبرده سوار\tبه جان گرانمایه اسفندیار\nکه نه هرگزت روی دشمن کنم\tنفرمایمت بد نه خود من کنم\nتو هرچ اندرین کار دانی بگوی\tکه تو چاره\u200cدانی و من چاره\u200cجوی\nخ

In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 42 ('چ')
  expected output: 39 ('و')
Step    1
  input: 39 ('و')
  expected output: 2 (' ')
Step    2
  input: 2 (' ')
  expected output: 14 ('ا')
Step    3
  input: 14 ('ا')
  expected output: 24 ('ز')
Step    4
  input: 24 ('ز')
  expected output: 2 (' ')


In [13]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024
dataset

<BatchDataset shapes: ((64, 1000), (64, 1000)), types: (tf.int64, tf.int64)>

# Model

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 1000, 48) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           12288     
_________________________________________________________________
gru_2 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
gru_3 (GRU)                  (64, None, 1024)          6297600   
_________________________________________________________________
dense_2 (Dense)              (64, None, 48)            49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [30]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 1000, 48)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       3.870893


In [0]:
model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy(from_logits=True), metrics=['accuracy'])

In [32]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
39/39 [==============================] - 49s 1s/step - loss: 3.2483 - accuracy: 0.1554
Epoch 2/10
39/39 [==============================] - 48s 1s/step - loss: 2.6013 - accuracy: 0.2688
Epoch 3/10
39/39 [==============================] - 49s 1s/step - loss: 2.4008 - accuracy: 0.3071
Epoch 4/10
39/39 [==============================] - 48s 1s/step - loss: 2.1903 - accuracy: 0.3548
Epoch 5/10
39/39 [==============================] - 49s 1s/step - loss: 2.0208 - accuracy: 0.4029
Epoch 6/10
39/39 [==============================] - 49s 1s/step - loss: 1.8735 - accuracy: 0.4446
Epoch 7/10
39/39 [==============================] - 49s 1s/step - loss: 1.7400 - accuracy: 0.4837
Epoch 8/10
39/39 [==============================] - 49s 1s/step - loss: 1.6189 - accuracy: 0.5222
Epoch 9/10
39/39 [==============================] - 48s 1s/step - loss: 1.5189 - accuracy: 0.5492
Epoch 10/10
39/39 [==============================] - 49s 1s/step - loss: 1.4493 - accuracy: 0.5677


In [33]:
main_model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
main_model.set_weights(model.get_weights())
model = main_model
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            12288     
_________________________________________________________________
gru_4 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
gru_5 (GRU)                  (1, None, 1024)           6297600   
_________________________________________________________________
dense_3 (Dense)              (1, None, 48)             49200     
Total params: 10,297,392
Trainable params: 10,297,392
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string, temperature = 1.0):
  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(idx2char[predicted_id])
  return (start_string + ''.join(text_generated))

In [38]:
print(generate_text(model, start_string=u"که ایران چو باغی ست خرم بهار", temperature=1))

که ایران چو باغی ست خرم بهار	برفت آمه‌ای جان تنکاه ستر
به ایرانشین را چنان سون شام	بتان مغان بنگرید از نژاد
جهان روی کشود نبرده بود	بدان کاردسرو میانار بر
همه بود پر ز مهان بکلواند	به نزدیک مامو به با چشم و باز
چه کور این سزاوش بیامد برش	بمالید بشکاوران گرمزار
همی گفت که کر که سیم و فزای	همی‌را مبای پدر غم دگر
همان ساخت ای خوبهر آزاد رست	بلو را ز دشمان کند تاگتر
شد اندر پدر سوی گیرین درفش	ز فرمود تا دو خماه و منند
ز هر کوقهان چلز گرددنکش خرد زال من
مرا گوش مایک بپرکند شاه	بهرآمه خداوند سالور چوسیاب
چو پوران خرا شادمان سربسر	چرا نامداران برو بارسا
نخستین هم اندر جهان سخن داد	که از اختر و گاه بر کاه تا
فرستید بسیه سواران بود	جنوی ز دو دشمن از کهار
ز بیناری بیژبند آورد	همر خونه بد تیرگاه انجمن
تو کوراگهٔ پیکسی چند آن	سوانش بیامد سواری بدو
به فرزند بد بوم و شگرین دست	یکی تاره دیدیم دامی بر خاک
چو آیی که از کشته نزدیک آورد	چگنها کس این برد رسمند شد
دگر گوهم رستم پول رستخت	نباید که هیمد سرافراز گفت
که لستش به بینشگه و اندر خود	همی خون برو آفرین باژیان
به روز گرابر به گرد و سخن	ز خشمند هرمس ک